In [1]:
from app.tools.youtube import fetch_comments
from app.tools.preprocess import select_fast_batch, preprocess_comments_df

df_all = fetch_comments("https://www.youtube.com/watch?v=26riTPNOJbc",
                       sqlite_path="./.cache.db",
                       include_replies=True,
                       max_comments=5000)

df_fast = select_fast_batch(df_all, mode="top_likes", limit=1200, include_replies=False)

df_pre, dbg = preprocess_comments_df(
    df_fast,
    min_chars=12,                 # було 20 → зробили 12
    keep_langs=("uk","ru","en","pl","cs","sk"),
    drop_spam=True,
    deduplicate=True,
    aggressive_stopword_check=False,
    return_debug=True
)

print(dbg.to_string(index=False))
print(df_pre[["comment_id","text_clean","lang","like_count"]].head())


 n_in  after_minlen  after_lang  after_spam  after_dedup  dropped_minlen  dropped_lang  dropped_spam  dropped_dup                                                                     lang_counts
   40            38          34          34           34               2             4             0            0 {'pl': 20, 'uk': 10, 'ru': 1, 'unknown': 1, 'sk': 1, 'hr': 1, 'en': 1, 'hu': 1}
                   comment_id  \
0  Ugx8YMgNr70XWO9_PGN4AaABAg   
1  UgwnH4Az1RliZfk5B7x4AaABAg   
2  UgxITgSgyx1a2BgJHd94AaABAg   
3  UgyPwsZ3OWS21CNg0AF4AaABAg   
4  Ugxa42Xs9RDHwgB3iJd4AaABAg   

                                          text_clean lang  like_count  
0  Чернівці люблю, чекаю на наступне відео) Вітаю...   uk          10  
1  W końcu doczekałem się nowego odcinka. Pozdrawiam   pl           9  
2  Щоб відчути справжню Україну у часи війни вам ...   uk           8  
3  Kowalski,nie pieprz bzdury,bo u was parkany ta...   pl           7  
4  Cześć, bardzo poruszające filmy. Jadąc z Karpa... 